In [33]:
import pandas as pd
import supabase
from supabase import create_client, Client
import plotly.express as px
import streamlit as st

In [34]:
def get_plays_data(table_name: str):
    data = supabase.table(table_name).select('*').execute().data
    df = pd.DataFrame(data)
    df['created_at'] = df['created_at'].apply(lambda x: x.split('.')[0])
    df['created_at'] = pd.to_datetime(df['created_at'])
    
    df['ses_start'] = df['created_at'] - pd.to_timedelta(df['duration'], unit='s')
    df['ses_end'] = df['created_at']
    
    df = df.drop(columns=['created_at'])
    return df[['id', 'ses_start', 'ses_end', 'duration', 'is_music']]

In [35]:
url: str = st.secrets["SUPABASE_URL"]
key: str = st.secrets["SUPABASE_KEY"]

supabase: Client = create_client(url, key)

In [36]:
df = get_plays_data('playTable')

In [38]:
df_daily = df.groupby([df['ses_start'].dt.date, 'is_music'])[['duration']].sum()

In [41]:
df_cumsum = df[['ses_end', 'duration', 'is_music']].set_index('ses_end').sort_index()
df_cumsum['duration_cumsum'] = df_cumsum.groupby('is_music')['duration'].cumsum()
df_cumsum

,duration,is_music,duration_cumsum
ses_end,,,
2023-04-28 20:27:17,4,False,4
2023-04-28 20:27:44,19,True,19
2023-04-29 11:21:34,16,True,35
2023-04-29 11:22:07,29,True,64
2023-04-29 11:23:15,5,False,9
